<a href="https://colab.research.google.com/github/Jeorr/ML/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

# Завантаження необхідних бібліотек
import tensorflow as tf
import numpy as np
import os
import gradio as gr
from tensorflow.keras.callbacks import ModelCheckpoint
import glob


# Завантаження даних
url = 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
path_to_file = tf.keras.utils.get_file('shakespeare.txt', url)

# Читання тексту
with open(path_to_file, 'r') as file:
    text = file.read()

text = ' '.join(text.split())  # Видаляє зайві пробіли

print(f'Довжина тексту: {len(text)} символів')
print(f'Приклад тексту:\n{text[:500]}')

# Унікальні символи у тексті
vocab = sorted(set(text))
print(f'Унікальних символів: {len(vocab)}')

# Створення словників символів
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)

# Векторизація тексту
text_as_int = np.array([char2idx[char] for char in text])

# Гіперпараметри
seq_length = 64  # Довжина послідовності
examples_per_epoch = len(text) // seq_length

# Створення тренувальних прикладів та міток
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Параметри для підготовки даних
BATCH_SIZE = 64
BUFFER_SIZE = 10000

# Створення тренувальної вибірки
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
)

# Створення моделі
vocab_size = len(vocab)  # Розмір словника
embedding_dim = 256  # Розмір векторного простору
rnn_units = 1024  # Кількість нейронів у RNN

# Шлях для збереження моделі
model_path = '/content/drive/MyDrive/ML/text_generation_model_v1.h5'
checkpoint_folder = "/content/drive/MyDrive/ML/checkpoints/text_generation_model_v1/"
checkpoint_path = checkpoint_folder + "{epoch:02d}.weights.h5"
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,  # Зберігайте лише ваги, щоб уникнути великих файлів
    save_freq='epoch',      # Зберігати після кожної епохи
    verbose=1
)

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return tf.keras.Sequential([
    tf.keras.layers.InputLayer(batch_input_shape=[batch_size, None]),
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True),
    tf.keras.layers.LSTM(units=rnn_units,return_sequences=True,stateful=True,recurrent_initializer=tf.keras.initializers.GlorotNormal()),
    tf.keras.layers.Dense(vocab_size)
])

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(
      y_true=labels,
      y_pred=logits,
      from_logits=True
    )

if os.path.exists(model_path):
    print("Завантаження збереженої моделі...")
    model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=BATCH_SIZE)
    model.load_weights(model_path)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
else:
    print("Модель не знайдена. Початок навчання...")
    model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=BATCH_SIZE)

    # Компіляція моделі
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    # Тренування
    EPOCHS = 110

    # Знаходимо останній чекпойнт
    latest_checkpoint = max(glob.glob(checkpoint_folder + "*.h5"), default=None)
    initial_epoch = 0;

    if latest_checkpoint:
        print(f"Завантаження моделі з чекпойнта {latest_checkpoint}")
        model.load_weights(latest_checkpoint)
        initial_epoch = int((latest_checkpoint.split('.')[0]).split('/')[-1])
    else:
        print("Чекпойнти не знайдено. Початок навчання з нуля.")

    history = model.fit(
        dataset,
        epochs=EPOCHS,
        initial_epoch=initial_epoch,
        callbacks=[checkpoint_callback]
    )

    # Збереження моделі
    model.save(model_path)
    print(f"Модель збережено як {model_path}")

# Завантаження збереженої моделі для генерації тексту
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(model_path)
model.build()

# Генерація тексту
def generate_text(start_string, num_generate=1000, temperature=1.0):
    # Перетворення стартового рядка в числовий вектор
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)  # Додаємо вимір для batch

    text_generated = []  # Список для збереження згенерованих символів

    # Генерація символів
    for _ in range(num_generate):
        # Передбачення наступного символу
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)  # Видаляємо вимір пакету
        predictions = predictions / temperature  # Налаштування температури

        # Вибір наступного символу на основі ймовірностей
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Оновлення вхідного рядка
        input_eval = tf.expand_dims([predicted_id], 0)

        # Додавання згенерованого символу до результату
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

# тестуємо
print(generate_text('Romeo: ', 500, 0.4))

# Створення інтерфейсу
gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(lines=2, placeholder="Введіть початковий текст...", label="Стартовий текст"),
        gr.Slider(100, 1000, step=100, label="Кількість символів"),
        gr.Slider(0.1, 2.0, step=0.1, value=1.0, label="Температура"),
    ],
    outputs="text",
    title="Генерація тексту на основі Шекспіра",
    description="Введіть стартовий текст і налаштуйте параметри генерації.",
).launch(debug=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Довжина тексту: 1108152 символів
Приклад тексту:
First Citizen: Before we proceed any further, hear me speak. All: Speak, speak. First Citizen: You are all resolved rather to die than to famish? All: Resolved. resolved. First Citize

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


266/266 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7285 - loss: 0.4898
Epoch 99: saving model to /content/drive/MyDrive/ML/checkpoints/text_generation_model_v1/99.weights.h5
266/266 ━━━━━━━━━━━━━━━━━━━━ 1411s 5s/step - accuracy: 0.7285 - loss: 0.4899
Epoch 100/110
266/266 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.7277 - loss: 0.4806
Epoch 100: saving model to /content/drive/MyDrive/ML/checkpoints/text_generation_model_v1/100.weights.h5
266/266 ━━━━━━━━━━━━━━━━━━━━ 1519s 6s/step - accuracy: 0.7277 - loss: 0.4806
Epoch 101/110
266/266 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7305 - loss: 0.4639
Epoch 101: saving model to /content/drive/MyDrive/ML/checkpoints/text_generation_model_v1/101.weights.h5
266/266 ━━━━━━━━━━━━━━━━━━━━ 1476s 5s/step - accuracy: 0.7305 - loss: 0.4639
Epoch 102/110
266/266 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7329 - loss: 0.4531
Epoch 102: saving model to /content/drive/MyDrive/ML/checkpoints/text_generation_model_v1/102.weights.h5
266/266 ━━━━━

Модель збережено як /content/drive/MyDrive/ML/text_generation_model_v1.h5
Romeo:  Now Lord joy &, I go; Unsued upon, , unto , let I'll Walk -bord &ubon, Knows love which &CIO: death, Zeep VINCENTIO: she affection xtremes? his last of King LUCIO: Come, be ar France: Grey post Berchance Flent, Such despair, Because quarrel QULIET: No, zen; Come --o, he y GREMIO: if Leccuse Warwick! . zents rise On Your ; Son A: Great her comes. Capulet! BENVOLIO: Strike, was Xuthonour, . Therefore $ose:  r : men ry maid, discontent to your grace of one known zell?  . murder OSPERO: Faith, come
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://caf8f004cfc265f921.gradio.live

This share link expires in 72 hours. For free